<center> <h1> House Price Pridiction </h1> </center>

<center><img src="image.gif"/></center>

# Business Problem
Pridict sales price of the house based on 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.


# 1. Importing Libraries

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# 2. Reading Data

## 2.1 Read csv

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
pd.set_option('display.max_columns', None)
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## 2.2 Datypes

### 2.2.1 Total of Datatypes

In [6]:
train.dtypes.value_counts()

object     43
int64      35
float64     3
dtype: int64

We can see that there are 43 Quantitative and 38 Qualitative variables. To find the extact data type we must explore the data further.

### 2.2.2 Types of Data

* Nominal: Attributes of a variable are differentiated only by name (category) and there is no order (rank, position).

* Ordinal: Attributes of a variable are differentiated by order (rank, position), but we do not know the relative degree of difference between them.

* Interval: Attributes of a variable are differentiated by the degree of difference between them, but there is no absolute zero, and the ratio between the attributes is unknown.

* Ratio: Attributes of a variable are differentiated by the degree of difference between them, there is absolute zero, and we could find the ratio between the attributes.

* Numerica and Continuous: Attributes can take any value in this range. The number of permitted values is uncountable.


In [7]:
nominal = ['MSSubClass' ,'MSZoning' ,'Street' ,'Alley' ,'LotShape' ,'Utilities' ,'LotConfig' ,'Neighborhood' ,'RoofStyle' ,'RoofMatl' ,'Exterior1st' ,'Exterior2nd' ,'MasVnrType' ,'Foundation' ,'Heating' ,'CentralAir' ,'Electrical' ,'GarageType' ,'MiscVal' ,'SaleType' ,'SaleCondition','MiscFeature']

ordinal = ['LandContour', 'LandSlope' ,'Condition1' ,'Condition2' ,'BldgType' ,'HouseStyle' ,'OverallQual' ,'OverallCond' ,'ExterQual' ,'ExterCond' ,'BsmtQual' ,'BsmtCond' ,'BsmtExposure' ,'BsmtFinType1' ,'HeatingQC' ,'FireplaceQu' ,'GarageFinish' ,'GarageQual' ,'GarageCond' ,'PoolQC' ,'Fence', 'BsmtFinType2', 'KitchenQual','Functional','PavedDrive']

continous = ['YearBuilt' , 'YearRemodAdd' , 'MasVnrArea' , '1stFlrSF' , '2ndFlrSF' , 'LowQualFinSF' , 'GrLivArea' , 'FullBath' , 'HalfBath' , 'BedroomAbvGr' , 'KitchenAbvGr' , 'Fireplaces' , 'GarageYrBlt' , 'GarageArea' ,'GarageCars' , 'WoodDeckSF' , 'OpenPorchSF' , 'EnclosedPorch' , '3SsnPorch' , 'ScreenPorch' , 'PoolArea' , 'MoSold' , 'YrSold','LotFrontage', 'LotArea', 'BsmtFinSF1','BsmtFinSF2','BsmtUnfSF', 'BsmtFullBath','TotalBsmtSF','BsmtHalfBath','TotRmsAbvGrd']

In [9]:
pd.set_option('display.max_columns', None)
train[nominal].head()

,MSSubClass,MSZoning,Street,Alley,LotShape,Utilities,LotConfig,Neighborhood,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,Heating,CentralAir,Electrical,GarageType,MiscVal,SaleType,SaleCondition,MiscFeature
0,60,RL,Pave,NaN,Reg,AllPub,Inside,CollgCr,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GasA,Y,SBrkr,Attchd,0,WD,Normal,NaN
1,20,RL,Pave,NaN,Reg,AllPub,FR2,Veenker,Gable,CompShg,MetalSd,MetalSd,None,CBlock,GasA,Y,SBrkr,Attchd,0,WD,Normal,NaN
2,60,RL,Pave,NaN,IR1,AllPub,Inside,CollgCr,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GasA,Y,SBrkr,Attchd,0,WD,Normal,NaN
3,70,RL,Pave,NaN,IR1,AllPub,Corner,Crawfor,Gable,CompShg,Wd Sdng,Wd Shng,None,BrkTil,GasA,Y,SBrkr,Detchd,0,WD,Abnorml,NaN
4,60,RL,Pave,NaN,IR1,AllPub,FR2,NoRidge,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GasA,Y,SBrkr,Attchd,0,WD,Normal,NaN


In [10]:
train[ordinal].head()

,LandContour,LandSlope,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,BsmtFinType2,KitchenQual,Functional,PavedDrive
0,Lvl,Gtl,Norm,Norm,1Fam,2Story,7,5,Gd,TA,Gd,TA,No,GLQ,Ex,NaN,RFn,TA,TA,NaN,NaN,Unf,Gd,Typ,Y
1,Lvl,Gtl,Feedr,Norm,1Fam,1Story,6,8,TA,TA,Gd,TA,Gd,ALQ,Ex,TA,RFn,TA,TA,NaN,NaN,Unf,TA,Typ,Y
2,Lvl,Gtl,Norm,Norm,1Fam,2Story,7,5,Gd,TA,Gd,TA,Mn,GLQ,Ex,TA,RFn,TA,TA,NaN,NaN,Unf,Gd,Typ,Y
3,Lvl,Gtl,Norm,Norm,1Fam,2Story,7,5,TA,TA,TA,Gd,No,ALQ,Gd,Gd,Unf,TA,TA,NaN,NaN,Unf,Gd,Typ,Y
4,Lvl,Gtl,Norm,Norm,1Fam,2Story,8,5,Gd,TA,Gd,TA,Av,GLQ,Ex,TA,RFn,TA,TA,NaN,NaN,Unf,Gd,Typ,Y


In [11]:
train[continous].head()

,YearBuilt,YearRemodAdd,MasVnrArea,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageYrBlt,GarageArea,GarageCars,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MoSold,YrSold,LotFrontage,LotArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,BsmtFullBath,TotalBsmtSF,BsmtHalfBath,TotRmsAbvGrd
0,2003,2003,196.0,856,854,0,1710,2,1,3,1,0,2003.0,548,2,0,61,0,0,0,0,2,2008,65.0,8450,706,0,150,1,856,0,8
1,1976,1976,0.0,1262,0,0,1262,2,0,3,1,1,1976.0,460,2,298,0,0,0,0,0,5,2007,80.0,9600,978,0,284,0,1262,1,6
2,2001,2002,162.0,920,866,0,1786,2,1,3,1,1,2001.0,608,2,0,42,0,0,0,0,9,2008,68.0,11250,486,0,434,1,920,0,6
3,1915,1970,0.0,961,756,0,1717,1,0,3,1,1,1998.0,642,3,0,35,272,0,0,0,2,2006,60.0,9550,216,0,540,1,756,0,7
4,2000,2000,350.0,1145,1053,0,2198,2,1,4,1,1,2000.0,836,3,192,84,0,0,0,0,12,2008,84.0,14260,655,0,490,1,1145,0,9


## 2. Missing Values

### 2.2.1 Check for missing values

In [13]:
# In Train set
nulls = train.isnull().sum() 
for x,y in enumerate(nulls):
    if y != 0:
        percent = nulls[x]/len(train)
        print(f'{train.columns[x]}:\t{nulls[x]}\tPercent: {round(percent * 100,2)}%')

LotFrontage:	259	Percent: 17.74%
Alley:	1369	Percent: 93.77%
MasVnrType:	8	Percent: 0.55%
MasVnrArea:	8	Percent: 0.55%
BsmtQual:	37	Percent: 2.53%
BsmtCond:	37	Percent: 2.53%
BsmtExposure:	38	Percent: 2.6%
BsmtFinType1:	37	Percent: 2.53%
BsmtFinType2:	38	Percent: 2.6%
Electrical:	1	Percent: 0.07%
FireplaceQu:	690	Percent: 47.26%
GarageType:	81	Percent: 5.55%
GarageYrBlt:	81	Percent: 5.55%
GarageFinish:	81	Percent: 5.55%
GarageQual:	81	Percent: 5.55%
GarageCond:	81	Percent: 5.55%
PoolQC:	1453	Percent: 99.52%
Fence:	1179	Percent: 80.75%
MiscFeature:	1406	Percent: 96.3%


In [15]:
# In Test set
nulls = test.isnull().sum() 
for x,y in enumerate(nulls):
    if y != 0:
        percent = nulls[x]/len(test)
        print(f'{test.columns[x]}:\t{nulls[x]}\tPercent: {round(percent * 100,2)}%')

MSZoning:	4	Percent: 0.27%
LotFrontage:	227	Percent: 15.56%
Alley:	1352	Percent: 92.67%
Utilities:	2	Percent: 0.14%
Exterior1st:	1	Percent: 0.07%
Exterior2nd:	1	Percent: 0.07%
MasVnrType:	16	Percent: 1.1%
MasVnrArea:	15	Percent: 1.03%
BsmtQual:	44	Percent: 3.02%
BsmtCond:	45	Percent: 3.08%
BsmtExposure:	44	Percent: 3.02%
BsmtFinType1:	42	Percent: 2.88%
BsmtFinSF1:	1	Percent: 0.07%
BsmtFinType2:	42	Percent: 2.88%
BsmtFinSF2:	1	Percent: 0.07%
BsmtUnfSF:	1	Percent: 0.07%
TotalBsmtSF:	1	Percent: 0.07%
BsmtFullBath:	2	Percent: 0.14%
BsmtHalfBath:	2	Percent: 0.14%
KitchenQual:	1	Percent: 0.07%
Functional:	2	Percent: 0.14%
FireplaceQu:	730	Percent: 50.03%
GarageType:	76	Percent: 5.21%
GarageYrBlt:	78	Percent: 5.35%
GarageFinish:	78	Percent: 5.35%
GarageCars:	1	Percent: 0.07%
GarageArea:	1	Percent: 0.07%
GarageQual:	78	Percent: 5.35%
GarageCond:	78	Percent: 5.35%
PoolQC:	1456	Percent: 99.79%
Fence:	1169	Percent: 80.12%
MiscFeature:	1408	Percent: 96.5%
SaleType:	1	Percent: 0.07%


We can see that test data has more missing values then our train data. We will impute the values based on the datatype of the feature.


### 2.2.2 Impute missing values

In [16]:
# First we will drop columns which have more then 500 missing values
null=train.loc[:,train.isnull().sum()>500]

train.drop(null,inplace=True,axis=1)
test.drop(null,inplace=True,axis=1)

In [17]:
# In Train set
nulls = train.isnull().sum() 
for x,y in enumerate(nulls):
    if y != 0:
        percent = nulls[x]/len(train)
        print(f'{train.columns[x]}:\t{nulls[x]}\tPercent: {round(percent * 100,2)}%')

LotFrontage:	259	Percent: 17.74%
MasVnrType:	8	Percent: 0.55%
MasVnrArea:	8	Percent: 0.55%
BsmtQual:	37	Percent: 2.53%
BsmtCond:	37	Percent: 2.53%
BsmtExposure:	38	Percent: 2.6%
BsmtFinType1:	37	Percent: 2.53%
BsmtFinType2:	38	Percent: 2.6%
Electrical:	1	Percent: 0.07%
GarageType:	81	Percent: 5.55%
GarageYrBlt:	81	Percent: 5.55%
GarageFinish:	81	Percent: 5.55%
GarageQual:	81	Percent: 5.55%
GarageCond:	81	Percent: 5.55%


In [18]:
nulls = test.isnull().sum() 
for x,y in enumerate(nulls):
    if y != 0:
        percent = nulls[x]/len(test)
        print(f'{test.columns[x]}:\t{nulls[x]}\tPercent: {round(percent * 100,2)}%')

MSZoning:	4	Percent: 0.27%
LotFrontage:	227	Percent: 15.56%
Utilities:	2	Percent: 0.14%
Exterior1st:	1	Percent: 0.07%
Exterior2nd:	1	Percent: 0.07%
MasVnrType:	16	Percent: 1.1%
MasVnrArea:	15	Percent: 1.03%
BsmtQual:	44	Percent: 3.02%
BsmtCond:	45	Percent: 3.08%
BsmtExposure:	44	Percent: 3.02%
BsmtFinType1:	42	Percent: 2.88%
BsmtFinSF1:	1	Percent: 0.07%
BsmtFinType2:	42	Percent: 2.88%
BsmtFinSF2:	1	Percent: 0.07%
BsmtUnfSF:	1	Percent: 0.07%
TotalBsmtSF:	1	Percent: 0.07%
BsmtFullBath:	2	Percent: 0.14%
BsmtHalfBath:	2	Percent: 0.14%
KitchenQual:	1	Percent: 0.07%
Functional:	2	Percent: 0.14%
GarageType:	76	Percent: 5.21%
GarageYrBlt:	78	Percent: 5.35%
GarageFinish:	78	Percent: 5.35%
GarageCars:	1	Percent: 0.07%
GarageArea:	1	Percent: 0.07%
GarageQual:	78	Percent: 5.35%
GarageCond:	78	Percent: 5.35%
SaleType:	1	Percent: 0.07%


#### 2.2.2.1 Train Set imputation

In [16]:
# Train Set imputation
# train['Alley'].fillna('No', inplace=True)
train['BsmtQual'].fillna('No',inplace=True)
train['BsmtCond'].fillna('No', inplace=True)
train['BsmtExposure'].fillna('No', inplace=True)
train['BsmtFinType2'].fillna('No', inplace=True)
# train['FireplaceQu'].fillna('No',inplace=True)
train['MasVnrType'].fillna('No',inplace=True)
train['GarageType'].fillna('No', inplace=True)
train['GarageFinish'].fillna('No', inplace=True)
train['GarageQual'].fillna('No', inplace=True)
train['GarageCond'].fillna('No', inplace=True)
# train['PoolQC'].fillna('No', inplace=True)
# train['Fence'].fillna('No', inplace=True)
# train['MiscFeature'].fillna('No', inplace=True)
train['BsmtFinType1'].fillna('No', inplace=True)

In [18]:
train['Electrical'].fillna(train.Electrical.mode()[0], inplace=True)

In [20]:
train['LotFrontage'].fillna(0, inplace=True)
train['GarageYrBlt'].fillna(0, inplace=True)
train['BsmtFinType1'].fillna(0, inplace=True)
train['MasVnrArea'].fillna(0, inplace=True)

#### 2.2.2.2 Test Set imputation

In [21]:
test['MSZoning'].fillna(train.MSZoning.mode()[0], inplace=True)
test['Utilities'].fillna(train.Utilities.mode()[0], inplace=True)
test['Exterior1st'].fillna(train.Exterior1st.mode()[0], inplace=True)
test['Exterior2nd'].fillna(train.Exterior2nd.mode()[0], inplace=True)
test['MasVnrType'].fillna(train.MasVnrType.mode()[0], inplace=True)
test['KitchenQual'].fillna(train.KitchenQual.mode()[0], inplace=True)
test['Functional'].fillna(train.Functional.mode()[0], inplace=True)
test['SaleType'].fillna(train.SaleType.mode()[0], inplace=True)

In [27]:
# test['Alley'].fillna('No', inplace=True)
test['MasVnrType'].fillna('No', inplace=True)
test['BsmtQual'].fillna('No', inplace=True)
test['BsmtCond'].fillna('No', inplace=True)
test['BsmtExposure'].fillna('No', inplace=True)
test['BsmtFinType1'].fillna('No', inplace=True)
test['BsmtFinType2'].fillna('No', inplace=True)
# test['FireplaceQu'].fillna('No', inplace=True)
test['GarageType'].fillna('No', inplace=True)
test['GarageQual'].fillna('No', inplace=True)
test['GarageCond'].fillna('No', inplace=True)
# test['PoolQC'].fillna('No', inplace=True)
# test['Fence'].fillna('No', inplace=True)
# test['MiscFeature'].fillna('No', inplace=True)
test['GarageFinish'].fillna('No', inplace=True)

In [28]:
test['LotFrontage'].fillna(0, inplace=True)
test['MasVnrArea'].fillna(0, inplace=True)
test['BsmtFinSF1'].fillna(0, inplace=True)
test['BsmtFinSF2'].fillna(0, inplace=True)
test['BsmtUnfSF'].fillna(0, inplace=True)
test['BsmtFullBath'].fillna(0, inplace=True)
test['BsmtHalfBath'].fillna(0, inplace=True)
test['GarageYrBlt'].fillna(0, inplace=True)
test['GarageCars'].fillna(0, inplace=True)
test['GarageArea'].fillna(0, inplace=True)
test['TotalBsmtSF'].fillna(0, inplace=True)
test['TotalBsmtSF'].fillna(0, inplace=True)

In [31]:
train.to_csv('train2.csv')
test.to_csv('test2.csv')